In [72]:
import pandas as pd
import json
from pathlib import Path 
from sqlalchemy import create_engine, Integer, String, Date, VARCHAR, text # import psycog2
from datetime import datetime,timedelta

In [73]:

print(dfs['manutencoes'].columns)



Index(['id_manutencao', 'id_maquina', 'data', 'tipo', 'custo', 'responsavel'], dtype='object')


In [ ]:
caminho_json = Path(r"C:\Users\lucas\OneDrive\Portfólio\Arquivo_Original\mine_dados.json")

#Ler o json
with open(caminho_json, 'r', encoding='utf-8-sig') as f:
    dados = json.load(f)

dfs = {tabela: pd.DataFrame(conteudo) for tabela, conteudo in dados.items()}

##Parâmetros de conexão
dbname =   'Port_Mineracao'
user =     'postgres'
password = '901012'
host =     'localhost'
port =     '5432'

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}")


for tabela, df in dfs.items():
    df.to_sql(tabela, engine, index=False, if_exists='replace')
    print(f"Tabela '{tabela}' Enviada com sucesso!")

query = """
SELECT 
    m.id_maquina,
    m.tipo AS tipo_maquina,
    m.status,
	STRING_AGG(DISTINCT ma.responsavel, ', ') AS responsaveis_manutencao,
	ma.tipo AS tipo_manutencao,
	COALESCE(i.gravidade, 'Sem ocorrências') AS nivel_gravidade,
	COUNT(i.id_incidente) AS total_incidentes,
    m.data_aquisicao,
    COALESCE(SUM(ma.custo), 0) AS custo_total_manutencao
	
FROM       maquinas    m
LEFT JOIN  manutencoes ma  ON m.id_maquina = ma.id_maquina
LEFT JOIN  incidentes  i   ON m.id_maquina = i.id_maquina
GROUP BY 
    m.id_maquina,
    m.tipo,
    m.status,
    m.data_aquisicao,
    ma.tipo,
    i.gravidade
ORDER BY 
m.id_maquina;

"""

# 3️⃣ Executar a query e carregar no pandas
kpis = pd.read_sql(query, engine)

# 4️⃣ Salvar em CSV para Power BI
kpis.to_csv("kpis_mineracao.csv", index=False, encoding='utf-8-sig')
print("KPIs exportados com sucesso para 'kpis_mineracao.csv'!")

# 5️⃣ Fechar conexão
engine.dispose()
print("Pipeline concluído com sucesso!")


Tabela 'maquinas' Enviada com sucesso!
Tabela 'operadores' Enviada com sucesso!
Tabela 'manutencoes' Enviada com sucesso!
Tabela 'incidentes' Enviada com sucesso!
KPIs exportados com sucesso para 'kpis_mineracao.csv'!
Pipeline concluído com sucesso!
